In [37]:
import pygame
import math
import random

In [38]:
# 2. Constantes do Jogo (Cores, Tamanhos, Velocidades, etc.)
# Cores Gerais (RGB)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0) # Cor do Pac-Man
# ... (outras cores para fantasmas, etc.)

# Configurações do Labirinto
CELL_SIZE = 25 # Tamanho de cada célula (bloco) em pixels
WALL_LINE_THICKNESS = 4
# ... (outras constantes de tamanho se necessário)

# Configurações do Pac-Man
PACMAN_RADIUS = int(CELL_SIZE * 0.4)
PACMAN_SPEED = 2 # Velocidade em pixels por frame

# Configurações da Pílula de Poder
POWER_PILL_DURATION = 8 * 1000 # Duração do efeito da pílula de poder em milissegundos (8 segundos)
POWER_PILL_SCORE = 50 # Pontos ganhos por comer uma pílula de poder

# Configurações dos Fantasmas (opcional, pode ser adicionado depois)
GHOST_RADIUS = int(CELL_SIZE * 0.4)
GHOST_SPEED_NORMAL = 2 
GHOST_SPEED_FRIGHTENED = 1 # Quando o Pacman comer a pílula

# Cores dos fantasmas (RGB)
# No Pac-Man original, são Blinky (vermelho), Pinky (rosa), Inky (ciano), Clyde (laranja)
GHOST_BLINKY_COLOR = (255, 0, 0)      # Vermelho
GHOST_PINKY_COLOR = (255, 192, 203)   # Rosa
GHOST_INKY_COLOR = (0, 255, 255)      # Ciano
GHOST_CLYDE_COLOR = (255, 165, 0)     # Laranja
GHOST_FRIGHTENED_COLOR = (0, 0, 255)  # Azul (quando assustado)
GHOST_EATEN_COLOR = (128, 128, 128)   # Cinza (quando está voltando para a casa)

# Posições iniciais dos fantasmas (coordenadas do grid)
# Essas são posições aproximadas da "casa" dos fantasmas no labirinto
GHOST_START_POSITIONS = {
    "blinky": (13, 13), # Blinky geralmente começa fora da casa ou na porta
    "pinky": (13, 14),  # Pinky no meio da casa
    "inky": (12, 14),   # Inky à esquerda da casa
    "clyde": (14, 14),  # Clyde à direita da casa
}


# Constantes para a Casa dos Fantasmas
# Estas são as coordenadas do grid que definem a ÁREA da casa dos fantasmas (incluindo porta e interior)
GHOST_HOUSE_DOOR_ROW = 12             # A linha Y do grid onde a porta (4) está localizada
GHOST_HOUSE_DOOR_COL_START = 13       # A coluna X inicial do grid da porta (4)
GHOST_HOUSE_DOOR_COL_END = 14         # A coluna X final do grid da porta (4)

GHOST_HOUSE_INTERIOR_MIN_X = 12       # Coluna inicial (X) da área interna de movimentação dos fantasmas (onde há '0's dentro da casa)
GHOST_HOUSE_INTERIOR_MAX_X = 15       # Coluna final (X) da área interna
GHOST_HOUSE_INTERIOR_MIN_Y = 13       # Linha inicial (Y) da área interna
GHOST_HOUSE_INTERIOR_MAX_Y = 14       # Linha final (Y) da área interna

In [39]:
# --- 3. Definição do Labirinto (Matriz 2D) ---
# Convenções para a matriz:
# 0 = Caminho vazio (onde o Pac-Man e fantasmas podem andar)
# 1 = Parede (obstáculo)
# 2 = Pílula (pequeno ponto comestível)
# 3 = Pílula de Poder (ponto grande, deixa fantasmas vulneráveis)
# 4 = Porta da casa dos fantasmas (apenas fantasmas podem atravessar)

level = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 4, 4, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1], # Linha da porta dos fantasmas
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], # Túnel lateral 
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1],
    [1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1],
    [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
    [1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1],
    [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]
# Calcular as dimensões da tela com base no labirinto
SCREEN_WIDTH = len(level[0]) * CELL_SIZE
SCREEN_HEIGHT = len(level) * CELL_SIZE

# ---Contagem de Pílulas e Cópia do Labirinto Original ---
original_level = [row[:] for row in level] 

total_pills = 244
print(f"DEBUG_INICIAL: Total de pílulas contadas no início: {total_pills}") # 244 (240 pílulas normais e 4 pílulas de poder).
# Variável global (ou gerenciada por uma classe de jogo) para as pílulas restantes
pills_left = total_pills # O número de pílulas no início do nível


DEBUG_INICIAL: Total de pílulas contadas no início: 244


In [40]:
# 4. Classes do Jogo (PacMan e Ghost)
# --- Classe PacMan ---
class PacMan:
    def __init__(self, start_x_grid, start_y_grid, radius, speed, cell_size):
        self.grid_x = start_x_grid
        self.grid_y = start_y_grid

        # Variáveis para a pílula de poder
        self.is_powered_up = False
        self.power_up_timer = 0 # Usaremos pygame.time.get_ticks() para gerenciar isso
        
        # Posição em pixels, centralizada na célula
        self.pixel_x = float(self.grid_x * cell_size + cell_size // 2)
        self.pixel_y = float(self.grid_y * cell_size + cell_size // 2)

        self.radius = radius
        self.color = YELLOW
        self.speed = speed
        self.cell_size = cell_size

        # Direção de movimento atual (pixels por frame)
        self.dx = 0
        self.dy = 0
        
        # Direção desejada (para viradas)
        self.desired_dx_val = 0 # Valor de -1, 0 ou 1
        self.desired_dy_val = 0 # Valor de -1, 0 ou 1


        # Animação da boca do Pac-Man
        self.mouth_open = True  # Começa com a boca aberta
        self.mouth_speed = 2.0  # Velocidade da animação da boca 
        self.current_mouth_angle = 45 # Ângulo inicial da boca aberta (metade do ângulo total da boca)
        self.target_mouth_angle = 45 # Ângulo que a boca tenta alcançar
        self.closed_mouth_angle = 0  # Ângulo da boca fechada
        self.open_mouth_angle = 45   # Ângulo da boca totalmente aberta (metade do ângulo total da boca)

        # Direção para o desenho da boca (0=right, 90=up, 180=left, 270=down)
        self.facing_angle = 0 # Começa virado para a direita (0 graus)
        
        self.last_moved_direction = (1, 0) # (dx, dy) para a última direção em que realmente se moveu


    def set_direction(self, dx_val, dy_val):
        """Define a direção desejada do Pac-Man."""
        self.desired_dx_val = dx_val
        self.desired_dy_val = dy_val

    def get_grid_pos(self):
        """Retorna a posição do Pac-Man no grid (inteiros)."""
        return int(self.pixel_x // self.cell_size), int(self.pixel_y // self.cell_size)

    def is_aligned_to_grid(self):
        """Verifica se o Pac-Man está aproximadamente no centro de uma célula."""
        # Margem de erro para alinhamento
        tolerance = self.speed // 2 

        # Calcula a diferença entre a posição pixel e o centro da célula atual
        center_x_of_current_grid = self.grid_x * self.cell_size + self.cell_size // 2
        center_y_of_current_grid = self.grid_y * self.cell_size + self.cell_size // 2
        
        # Verifica se o pixel_x está próximo do centro horizontal da célula
        aligned_x = abs(self.pixel_x - center_x_of_current_grid) <= tolerance
        # Verifica se o pixel_y está próximo do centro vertical da célula
        aligned_y = abs(self.pixel_y - center_y_of_current_grid) <= tolerance

        return aligned_x and aligned_y


    def can_move_in_direction(self, dx_check, dy_check, game_level):
        """
        Verifica se é possível mover para a próxima célula na direção (dx_check, dy_check).
        Considera a próxima célula inteira.
        """
        current_grid_x, current_grid_y = self.get_grid_pos()

        target_grid_x = current_grid_x + dx_check
        target_grid_y = current_grid_y + dy_check


        # --- LÓGICA PARA TÚNEL: Permite movimento para fora do grid para teletransporte ---
        tunnel_row = 15 # <--- Defina esta constante para a linha do túnel (índice 15)

        # Se o Pac-Man está na linha do túnel E está tentando se mover para fora da tela
        if current_grid_y == tunnel_row:
            if (current_grid_x == 0 and dx_check < 0) or \
               (current_grid_x == len(game_level[0]) - 1 and dx_check > 0):
                return True # Permite o movimento "off-screen" para o túnel
    
        # Garante que a próxima célula está dentro dos limites do labirinto
        if not (0 <= target_grid_y < len(game_level) and 0 <= target_grid_x < len(game_level[0])):
            return False # Fora dos limites, não pode mover

        # Verifica se a próxima célula é uma parede (1)
        if game_level[target_grid_y][target_grid_x] == 1:
            return False # É uma parede, não pode mover
        
        # Adicione lógica para a porta dos fantasmas (4)
        # Pac-Man não pode atravessar a porta dos fantasmas
        if game_level[target_grid_y][target_grid_x] == 4:
            return False

        return True # Pode mover

    def update(self, game_level):
        """Atualiza a posição do Pac-Man e lida com colisões, viradas, e animação."""
        
        # 1. Atualizar a posição do Pac-Man no grid (sempre no início do update)
        self.grid_x = int(self.pixel_x // self.cell_size)
        self.grid_y = int(self.pixel_y // self.cell_size)

        points_gained = 0 # Inicializa a pontuação ganha neste frame
        pill_eaten_this_frame = False # Para saber se comeu algo
        
        # --- DEPURAÇÃO CRÍTICA: Posição do Pac-Man (SEMPRE ATIVO) ---
        # print(f"PacMan Grid: ({self.grid_x}, {self.grid_y}) | Pixel: ({self.pixel_x:.2f}, {self.pixel_y:.2f}) | DX: {self.dx}, DY: {self.dy}")

        # 2. Lógica de virada e movimento (com base na desired_direction)
        if self.desired_dx_val != 0 or self.desired_dy_val != 0:
            if self.is_aligned_to_grid():
                if self.can_move_in_direction(self.desired_dx_val, self.desired_dy_val, game_level):
                    self.dx = self.desired_dx_val * self.speed
                    self.dy = self.desired_dy_val * self.speed
                    self.pixel_x = float(self.grid_x * self.cell_size + self.cell_size // 2)
                    self.pixel_y = float(self.grid_y * self.cell_size + self.cell_size // 2)
                else:
                    self.desired_dx_val = 0
                    self.desired_dy_val = 0

        # 3. Aplicar movimento atual e tratar colisão com paredes
        if self.dx != 0 or self.dy != 0:
            if self.can_move_in_direction(self.dx // self.speed, self.dy // self.speed, game_level):
                self.pixel_x += self.dx
                self.pixel_y += self.dy
            else:
                self.dx = 0
                self.dy = 0
                self.pixel_x = float(self.grid_x * self.cell_size + self.cell_size // 2)
                self.pixel_y = float(self.grid_y * self.cell_size + self.cell_size // 2)

            # 4. Lógica de Consumo de Pílulas e Pílulas de Poder (Verifica na célula atual)
            # Re-calcula a posição no grid APÓS o movimento de pixel, para pegar a célula para a qual ele acabou de se mover
            current_grid_x_after_move = int(self.pixel_x // self.cell_size)
            current_grid_y_after_move = int(self.pixel_y // self.cell_size)

            # Garante que a posição está dentro dos limites do labirinto (especialmente útil para túneis)
            if 0 <= current_grid_y_after_move < len(game_level) and 0 <= current_grid_x_after_move < len(game_level[0]):
                cell_content_before_consumption = game_level[current_grid_y_after_move][current_grid_x_after_move] # <--- PEGA O CONTEÚDO ANTES DE MUDAR

                # Se a célula contém uma pílula ou pílula de poder
                if cell_content_before_consumption == 2 or cell_content_before_consumption == 3:
                    # Remove a pílula do labirinto (sempre)
                    game_level[current_grid_y_after_move][current_grid_x_after_move] = 0  
                    
                    # Define que uma pílula foi comida neste frame
                    pill_eaten_this_frame = True 

                    # Atribui os pontos e o estado de poder
                    if cell_content_before_consumption == 2:  # Se for uma pílula normal
                        points_gained = 10
                        
                    elif cell_content_before_consumption == 3:  # Se for uma pílula de poder
                        self.is_powered_up = True
                        self.power_up_timer = pygame.time.get_ticks()
                        points_gained = POWER_PILL_SCORE
                        print(f"Pac-Man Powered Up! Tempo: {self.power_up_timer}")
                    
            # --- Lógica: Passagem pelo Túnel Lateral ---
            tunnel_row = 15 # A linha do grid onde o túnel está
    
            # --- DEPURAÇÃO: Verificando condições do túnel (agora ativadas se grid_y for 15) ---
            if self.grid_y == tunnel_row: # Se Pac-Man está na linha do túnel
                # print(f"Tunnel Check: PacMan na linha {self.grid_y} (esperado {tunnel_row})") # DEPURAÇÃO ADICIONAL
                if self.grid_x == 0 and self.dx < 0: # Se está na coluna 0 e tentando ir para a esquerda
                    self.pixel_x = float((len(game_level[0]) - 1) * self.cell_size + self.cell_size // 2)
                    self.grid_x = len(game_level[0]) - 1
                    print("TELEPORTE: Esquerda para Direita!")
                elif self.grid_x == len(game_level[0]) - 1 and self.dx > 0: # Se está na coluna 27 e tentando ir para a direita
                    self.pixel_x = float(0 * self.cell_size + self.cell_size // 2)
                    self.grid_x = 0
                    print("TELEPORTE: Direita para Esquerda!")
            # FIM DA LÓGICA DO TÚNEL
                    
            # 5. Lógica de Animação da Boca
            self.last_moved_direction = (self.dx // self.speed, self.dy // self.speed) # Normaliza

            if self.mouth_open:
                self.current_mouth_angle -= self.mouth_speed * 2
                if self.current_mouth_angle <= self.closed_mouth_angle:
                    self.mouth_open = False
            else:
                self.current_mouth_angle += self.mouth_speed * 2
                if self.current_mouth_angle >= self.open_mouth_angle:
                    self.mouth_open = True
        else: # Pac-Man está parado
            if self.current_mouth_angle > self.closed_mouth_angle:
                self.current_mouth_angle -= self.mouth_speed * 2
                if self.current_mouth_angle <= self.closed_mouth_angle:
                    self.current_mouth_angle = self.closed_mouth_angle

        # 6. Atualiza o ângulo de rotação da boca (sempre, mesmo parado)
        if self.last_moved_direction == (1, 0): # Direita
            self.facing_angle = 0
        elif self.last_moved_direction == (-1, 0): # Esquerda
            self.facing_angle = 180
        elif self.last_moved_direction == (0, -1): # Cima
            self.facing_angle = 90
        elif self.last_moved_direction == (0, 1): # Baixo
            self.facing_angle = 270

        # 7. Gerenciar o temporizador da Pílula de Poder (sempre, mesmo parado)
        was_powered_up = self.is_powered_up 
        if self.is_powered_up:
            current_time = pygame.time.get_ticks()
            if current_time - self.power_up_timer > POWER_PILL_DURATION:
                self.is_powered_up = False
                self.power_up_timer = 0
                print("Pac-Man Power-Up Acabou!")

        # 8. Retornar os pontos ganhos
        return points_gained, pill_eaten_this_frame

    def draw(self, screen):
        """Desenha o Pac-Man na tela com animação da boca."""
        
        # Desenha o Pac-Man como um círculo (corpo)
        pygame.draw.circle(screen, self.color, (int(self.pixel_x), int(self.pixel_y)), self.radius)

        # Desenha o "buraco" da boca como um triângulo preto
        # (Lembrete: 'import math' deve estar no topo do seu arquivo)
        
        # Ponto 1 (centro do Pac-Man)
        point1 = (int(self.pixel_x), int(self.pixel_y))
        
        # Ponto 2 (borda superior da boca, ajustada pela direção)
        angle_rad_top = math.radians(self.facing_angle + self.current_mouth_angle)
        point2 = (int(self.pixel_x + self.radius * math.cos(angle_rad_top)),
                  int(self.pixel_y - self.radius * math.sin(angle_rad_top))) # -sin porque Y cresce para baixo
        
        # Ponto 3 (borda inferior da boca, ajustada pela direção)
        angle_rad_bottom = math.radians(self.facing_angle - self.current_mouth_angle)
        point3 = (int(self.pixel_x + self.radius * math.cos(angle_rad_bottom)),
                  int(self.pixel_y - self.radius * math.sin(angle_rad_bottom))) # -sin porque Y cresce para baixo
        
        # Desenha o triângulo preto que simula a boca aberta
        pygame.draw.polygon(screen, BLACK, [point1, point2, point3])
    


In [41]:
# --- Classe Ghost --- 
class Ghost:
    def __init__(self, name, start_x_grid, start_y_grid, radius, speed, color, cell_size):
        self.name = name # Nome do fantasma (ex: "blinky", "pinky")
        self.grid_x = start_x_grid
        self.grid_y = start_y_grid
        
        # Posição em pixels, centralizada na célula
        self.pixel_x = float(self.grid_x * cell_size + cell_size // 2)
        self.pixel_y = float(self.grid_y * cell_size + cell_size // 2)
        
        self.radius = radius
        self.speed_normal = speed # Velocidade normal
        self.speed_frightened = GHOST_SPEED_FRIGHTENED # Velocidade quando assustado (definida nas constantes)
        self.current_speed = speed # Velocidade atual
        self.color_normal = color # Cor original do fantasma
        self.color_frightened = GHOST_FRIGHTENED_COLOR # Cor azul quando assustado
        self.color_eaten = GHOST_EATEN_COLOR # Cor cinza quando voltando para a casa
        self.current_color = color # Cor que está sendo exibida
        self.cell_size = cell_size

        # Alvo de dispersão para este fantasma
        self.scatter_target = SCATTER_TARGETS[name] # Atribui o alvo de dispersão com base no nome
        self.target_mode = "scatter" # Ou "chase", "frightened", "eaten"
        # Estado do fantasma
        self.is_frightened = False # True quando Pac-Man come pílula de poder
        self.is_eaten = False      # True quando Pac-Man come o fantasma assustado (volta para a casa)
        
        # Direção de movimento (ainda não implementado)
        self.dx = 0
        self.dy = 0
        self.last_moved_direction = (0, 0) # Última direção que se moveu

        # Controle de movimento e evitar que fiquem presos
        self.target_grid_x = start_x_grid
        self.target_grid_y = start_y_grid
        self.moving_to_target = False

    def get_grid_pos(self):
        """Retorna a posição do fantasma no grid (inteiros)."""
        return int(self.pixel_x // self.cell_size), int(self.pixel_y // self.cell_size)
   
    def can_move_in_direction(self, dx_check, dy_check, game_level):
        current_grid_x, current_grid_y = self.get_grid_pos()
        target_grid_x = current_grid_x + dx_check
        target_grid_y = current_grid_y + dy_check

        # Obter as dimensões do labirinto
        maze_width = len(game_level[0])
        maze_height = len(game_level)

        # 1. Lógica de Túnel (Fantasmas também usam)
        tunnel_row = 15
        if current_grid_y == tunnel_row:
            # Se está tentando mover para fora da borda esquerda (coluna 0)
            if current_grid_x == 0 and dx_check < 0:
                return True
            # Se está tentando mover para fora da borda direita (última coluna)
            elif current_grid_x == maze_width - 1 and dx_check > 0:
                return True
        
        # --- NOVO: Verificação de limites para a POSIÇÃO ATUAL ---
        # Se a posição atual do fantasma está fora dos limites do labirinto (acontece no túnel antes do teletransporte)
        if not (0 <= current_grid_y < maze_height and 0 <= current_grid_x < maze_width):
            # Se ele não está na linha do túnel ou não está na coluna de saída do túnel,
            # então ele está em uma posição inválida e não pode mover.
            # Se for túnel, já foi retornado True acima.
            return False 
        
        current_cell_type = game_level[current_grid_y][current_grid_x] # Agora esta linha é segura.
        
        # 2. Verificar limites gerais do labirinto para a CELULA ALVO
        if not (0 <= target_grid_y < maze_height and 0 <= target_grid_x < maze_width):
            return False

        target_cell_type = game_level[target_grid_y][target_grid_x]

        # 3. Não pode mover para paredes sólidas (1)
        if target_cell_type == 1:
            return False

        # --- 4. Lógica da Casa dos Fantasmas (Porta 4 e Interior 0) ---

        # Define se a CÉLULA ALVO é parte da ZONA DA CASA (Porta 4 ou Interior 0)
        is_target_in_house_zone = (
            (target_grid_y == GHOST_HOUSE_DOOR_ROW and 
             GHOST_HOUSE_DOOR_COL_START <= target_grid_x <= GHOST_HOUSE_DOOR_COL_END) or # É a porta (4)
            (GHOST_HOUSE_INTERIOR_MIN_Y <= target_grid_y <= GHOST_HOUSE_INTERIOR_MAX_Y and 
             GHOST_HOUSE_INTERIOR_MIN_X <= target_grid_x <= GHOST_HOUSE_INTERIOR_MAX_X and 
             target_cell_type == 0) # É um '0' dentro da área da casa
        )
        
        # Define se a CÉLULA ATUAL é parte da ZONA DA CASA (Porta 4 ou Interior 0)
        is_current_in_house_zone = (
            (current_grid_y == GHOST_HOUSE_DOOR_ROW and 
             GHOST_HOUSE_DOOR_COL_START <= current_grid_x <= GHOST_HOUSE_DOOR_COL_END) or
            (GHOST_HOUSE_INTERIOR_MIN_Y <= current_grid_y <= GHOST_HOUSE_INTERIOR_MAX_Y and 
             GHOST_HOUSE_INTERIOR_MIN_X <= current_grid_x <= GHOST_HOUSE_INTERIOR_MAX_X and 
             current_cell_type == 0)
        )

        # Fantasmas COMIDOS (is_eaten=True) podem ir para qualquer lugar (incluindo casa/porta)
        if self.is_eaten:
            return True 

        # Fantasmas NORMAIS/ASSUSTADOS (is_eaten=False)
        else: # not self.is_eaten
            # Regra 1: Bloquear RE-ENTRADA na casa ou na porta (se vindo de FORA dela)
            if is_target_in_house_zone:
                if not is_current_in_house_zone: # Se a célula ATUAL NÃO é da zona da casa
                    return False # Bloqueia a re-entrada de fantasmas não comidos
            
            # Regra 2: Permitir SAÍDA da casa (de dentro para fora)
            # Se o fantasma está DENTRO da zona da casa E o alvo NÃO é da zona da casa
            if is_current_in_house_zone and not is_target_in_house_zone:
                # E o movimento é para cima (direção de saída da porta)
                if dy_check < 0: # Tentando mover para cima (direção de saída comum)
                    # Verifica se o alvo é o corredor principal acima da porta (linha 11).
                    if target_grid_y == (GHOST_HOUSE_DOOR_ROW - 1) and target_cell_type == 0:
                        return True # Permite a saída
                # Se não é para cima para o corredor, bloqueia (ex: tentar sair pelos lados ou para baixo)
                return False 
            
            # Regra 3: Permitir MOVIMENTO INTERNO na casa (se já está dentro)
            # Se o fantasma está dentro da zona da casa E o alvo também é da zona da casa
            if is_current_in_house_zone and is_target_in_house_zone:
                return True 

        return True # Permite movimento por padrão se nenhuma regra bloqueou antes.

        return True # Permite movimento por padrão se nenhuma regra bloqueou antes.


    def draw(self, screen):
        """Desenha o fantasma na tela."""
        # Por simplicidade, vamos desenhar como um círculo com uma "base" retangular.
        # Mais tarde, você pode usar imagens (sprites) para um visual autêntico.
        
        # Corpo do fantasma (círculo)
        pygame.draw.circle(screen, self.current_color, (int(self.pixel_x), int(self.pixel_y)), self.radius)
        
        # Base do fantasma (retângulo para simular a parte de baixo)
        # Ajusta a posição Y para que a base comece um pouco abaixo do centro do círculo
        rect_height = int(self.radius * 0.8) # Altura da base
        rect_y = int(self.pixel_y) # Começa no centro Y do círculo
        pygame.draw.rect(screen, self.current_color, 
                         (int(self.pixel_x - self.radius), rect_y, 
                          self.radius * 2, rect_height))
        
        # Olhos (apenas um esboço, você pode estilizar mais tarde)
        eye_radius = int(self.radius * 0.2)
        eye_offset = int(self.radius * 0.3)
        pygame.draw.circle(screen, WHITE, (int(self.pixel_x - eye_offset), int(self.pixel_y - eye_offset)), eye_radius)
        pygame.draw.circle(screen, BLACK, (int(self.pixel_x - eye_offset), int(self.pixel_y - eye_offset)), int(eye_radius * 0.5))
        pygame.draw.circle(screen, WHITE, (int(self.pixel_x + eye_offset), int(self.pixel_y - eye_offset)), eye_radius)
        pygame.draw.circle(screen, BLACK, (int(self.pixel_x + eye_offset), int(self.pixel_y - eye_offset)), int(eye_radius * 0.5))


    def update(self, game_level):
        """Atualiza a lógica de movimento do fantasma e seu estado visual."""
        
        # 1. Atualizar cor e velocidade com base no estado
        if self.is_frightened:
            self.current_color = self.color_frightened
            self.current_speed = self.speed_frightened
        elif self.is_eaten:
            self.current_color = self.color_eaten
            self.current_speed = self.speed_normal
        else:
            self.current_color = self.color_normal
            self.current_speed = self.speed_normal

        # 2. Capturar posição inicial do grid (do frame anterior) para a lógica de alinhamento
        initial_grid_x = int(self.pixel_x // self.cell_size)
        initial_grid_y = int(self.pixel_y // self.cell_size)

        # 3. LÓGICA DE MOVIMENTO ALEATÓRIO BÁSICO
        tolerance = self.current_speed / 2
        # Verifica alinhamento com o centro da célula atual para decidir nova direção
        aligned_x = abs(self.pixel_x - (initial_grid_x * self.cell_size + self.cell_size // 2)) <= tolerance
        aligned_y = abs(self.pixel_y - (initial_grid_y * self.cell_size + self.cell_size // 2)) <= tolerance

        if aligned_x and aligned_y: # Se alinhado, escolhe NOVO dx/dy
            # Snaps pixel_x/y para o centro exato da célula atual
            self.pixel_x = float(initial_grid_x * self.cell_size + self.cell_size // 2)
            self.pixel_y = float(initial_grid_y * self.cell_size + self.cell_size // 2)

            # Seleciona uma nova direção aleatória válida
            possible_directions = []
            directions = [(1, 0), (-1, 0), (0, 1), (0, -1)] # Direita, Esquerda, Baixo, Cima

            # Define o alvo do fantasma com base no modo (por enquanto, apenas scatter)
            target_x, target_y = self.scatter_target # Alvo para a fase de dispersão

            # Lista de (distancia, dx, dy) para escolher a melhor direção
            best_direction = None
            min_distance = float('inf')

            for d_x, d_y in directions:
                # Evita virar 180 graus, a menos que não haja outra opção
                if (d_x, d_y) != (-self.last_moved_direction[0], -self.last_moved_direction[1]) or len(possible_directions) == 0:
                    # can_move_in_direction é chamado com (d_x, d_y) a partir da posição *atual*
                    if self.can_move_in_direction(d_x, d_y, game_level):
                        possible_directions.append((d_x, d_y))
            
            if possible_directions:
                new_dx_val, new_dy_val = random.choice(possible_directions)
                self.dx = new_dx_val * self.current_speed
                self.dy = new_dy_val * self.current_speed
                self.last_moved_direction = (new_dx_val, new_dy_val)
            else: # Se não há onde ir (preso), para
                self.dx = 0
                self.dy = 0

        # 4. APLICA MOVIMENTO (aqui o pixel_x/y se move, potencialmente para fora da tela)
        self.pixel_x += self.dx
        self.pixel_y += self.dy

        # 5. ATUALIZA grid_x/y IMEDIATAMENTE APÓS O MOVIMENTO DE PIXEL
        # Isso garante que grid_x/y reflita a nova posição para a lógica do túnel.
        self.grid_x = int(self.pixel_x // self.cell_size)
        self.grid_y = int(self.pixel_y // self.cell_size)
        
        # 6. LÓGICA DE TÚNEL: Teletransporte IMEDIATO se saiu da tela
        maze_width = len(game_level[0])
        maze_height = len(game_level) # Para consistência, embora não usado na lógica do túnel Y aqui.

        tunnel_row = 15 
        
        # Verifica se o fantasma está na linha do túnel E se a coordenada grid_x saiu dos limites
        if self.grid_y == tunnel_row:
            if self.grid_x < 0: # Saiu pela esquerda (grid_x se tornou -1 ou menos)
                self.pixel_x = float((maze_width - 1) * self.cell_size + self.cell_size // 2)
                self.grid_x = maze_width - 1
            elif self.grid_x >= maze_width: # Saiu pela direita (grid_x se tornou maze_width ou mais)
                self.pixel_x = float(0 * self.cell_size + self.cell_size // 2)
                self.grid_x = 0

        # Nada mais abaixo deste ponto que chame can_move_in_direction ou acesse game_level
        # usando as posições de grid antes do próximo frame.


In [42]:
# 5. Inicialização do Pygame
pygame.init()
pygame.font.init()
game_font = pygame.font.Font(None, 24)
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Pac-Man em Python")
clock = pygame.time.Clock() # Para controlar o FPS

In [43]:
# 6. Criação de Instâncias dos Objetos do Jogo
# Crie seu Pac-Man
pacman = PacMan(14, 22, PACMAN_RADIUS, PACMAN_SPEED, CELL_SIZE)
score = 0
last_score_rendered = -1 # <--- Variável para controlar a renderização do score

#  --- Criação dos Fantasmas ---
ghosts = []
ghosts.append(Ghost("blinky", GHOST_START_POSITIONS["blinky"][0], GHOST_START_POSITIONS["blinky"][1], 
                    GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_BLINKY_COLOR, CELL_SIZE))
ghosts.append(Ghost("pinky", GHOST_START_POSITIONS["pinky"][0], GHOST_START_POSITIONS["pinky"][1], 
                    GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_PINKY_COLOR, CELL_SIZE))
ghosts.append(Ghost("inky", GHOST_START_POSITIONS["inky"][0], GHOST_START_POSITIONS["inky"][1], 
                   GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_INKY_COLOR, CELL_SIZE))
ghosts.append(Ghost("clyde", GHOST_START_POSITIONS["clyde"][0], GHOST_START_POSITIONS["clyde"][1], 
                    GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_CLYDE_COLOR, CELL_SIZE))

# --- Lógica de Reinício de Nível ---
def reset_level():
    global level, pills_left, score, pacman, original_level # <--- Usa 'global' para modificar variáveis globais
    
    # 1. Redefinir o labirinto para o estado original
    level = [row[:] for row in original_level] # Copia o labirinto original de volta
    
    # 2. Redefinir a contagem de pílulas
    pills_left = total_pills 
    
    # 3. Redefinir a posição do Pac-Man
    # (Você pode querer uma posição inicial diferente para cada nível ou sempre a mesma)
    pacman.pixel_x = float(1 * CELL_SIZE + CELL_SIZE // 2) # Posição inicial Pac-Man
    pacman.pixel_y = float(3 * CELL_SIZE + CELL_SIZE // 2)
    pacman.dx = 0
    pacman.dy = 0
    pacman.desired_dx_val = 0
    pacman.desired_dy_val = 0
    pacman.is_powered_up = False
    pacman.power_up_timer = 0
    pacman.mouth_open = True # Reseta a boca para aberta
    pacman.current_mouth_angle = pacman.open_mouth_angle
    pacman.facing_angle = 0
    pacman.last_moved_direction = (1, 0)

    ghosts.clear() # Limpa a lista atual de fantasmas
    ghosts.append(Ghost("blinky", GHOST_START_POSITIONS["blinky"][0], GHOST_START_POSITIONS["blinky"][1], 
                        GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_BLINKY_COLOR, CELL_SIZE))
    ghosts.append(Ghost("pinky", GHOST_START_POSITIONS["pinky"][0], GHOST_START_POSITIONS["pinky"][1], 
                        GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_PINKY_COLOR, CELL_SIZE))
    ghosts.append(Ghost("inky", GHOST_START_POSITIONS["inky"][0], GHOST_START_POSITIONS["inky"][1], 
                       GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_INKY_COLOR, CELL_SIZE))
    ghosts.append(Ghost("clyde", GHOST_START_POSITIONS["clyde"][0], GHOST_START_POSITIONS["clyde"][1], 
                        GHOST_RADIUS, GHOST_SPEED_NORMAL, GHOST_CLYDE_COLOR, CELL_SIZE))

    print("Nível Reiniciado! Todas as pílulas consumidas.")

In [44]:
# 7. Loop Principal do Jogo
running = True
while running:
    # 7.1. Processamento de Eventos (Entrada do Usuário, Fechar Janela)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False # Encerra o loop principal
        if event.type == pygame.KEYDOWN:
            # Controle do Pac-Man com as setas
            if event.key == pygame.K_LEFT:
                pacman.set_direction(-1, 0)
            elif event.key == pygame.K_RIGHT:
                pacman.set_direction(1, 0)
            elif event.key == pygame.K_UP:
                pacman.set_direction(0, -1)
            elif event.key == pygame.K_DOWN:
                pacman.set_direction(0, 1)

    # 7.2. Atualização do Estado do Jogo (Lógica do Jogo)
    points_gained, pill_eaten_this_frame = pacman.update(level) # Pega os pontos retornados pelo PacMan.update()
    
    if points_gained > 0:
        score += points_gained  # Adiciona ao placar total
        
    if pill_eaten_this_frame: # Se alguma pílula foi comida neste frame
        pills_left -= 1 # Decrementa a contagem de pílulas restantes
        # print(f"DEBUG: Pílula comida! Pílulas restantes: {pills_left}")
        
    # --- NOVO: DEPURAÇÃO PARA ENCONTRAR PÍLULAS RESTANTES ---
    current_pills_in_maze = 0
    remaining_pill_coords = [] # Lista para armazenar as coordenadas das pílulas
    for r_idx, row in enumerate(level):
        for c_idx, cell in enumerate(row):
            if cell == 2 or cell == 3:
                current_pills_in_maze += 1
                remaining_pill_coords.append((c_idx, r_idx)) # Adiciona (coluna, linha)
    
    # Imprime informações detalhadas apenas quando há poucas pílulas restantes
    #if current_pills_in_maze > 0 and current_pills_in_maze <= 5: # Ajuste o "5" para ver mais ou menos
        # print(f"DEBUG_FINAL_PILLS: Pílulas na matriz: {current_pills_in_maze}. Coordenadas: {remaining_pill_coords}")
    
    # Opcional: Se houver uma discrepância entre o contador e o que o maze mostra
    # if pills_left != current_pills_in_maze:
    #    print(f"!!! DISCREPÂNCIA: pills_left = {pills_left} mas maze tem {current_pills_in_maze} pílulas. !!!")

    # FIM DA DEPURAÇÃO DE PÍLULAS RESTANTES 
    # --- Verifica Condição de Reinício do Nível ---
    if pills_left <= 0:
        print("DEBUG: Condição de reinício atingida (pills_left <= 0)!")
        reset_level() # Chama a função para reiniciar o nível
        
    # ---  Atualizar e Desenhar Fantasmas ---
    for ghost in ghosts:
        ghost.update(level) # Passa o labirinto para o fantasma (para futuras lógicas de movimento/colisão)

    # --- NOVO: Lógica de Estado Assustado dos Fantasmas ---
    # Isso precisa ser feito APÓS o update do Pac-Man, pois depende do estado dele.
    for ghost in ghosts:
        # Se Pac-Man está powered up, e o fantasma NÃO estava assustado ou comido
        if pacman.is_powered_up and not ghost.is_eaten:
            # Ativa o estado assustado no fantasma
            if not ghost.is_frightened: # Só inverte a direção se acabou de ficar assustado
                ghost.is_frightened = True
                ghost.current_speed = GHOST_SPEED_FRIGHTENED # Garante a velocidade
                # Lógica para inverter a direção do fantasma
                ghost.dx *= -1
                ghost.dy *= -1
                ghost.last_moved_direction = (ghost.dx // ghost.current_speed, ghost.dy // ghost.current_speed)
                print(f"Fantasma {ghost.name} ficou assustado e inverteu a direção!")

        # Se Pac-Man NÃO está powered up, o fantasma não deve estar assustado (a menos que esteja comido)
        elif not pacman.is_powered_up:
            if ghost.is_frightened: # Se o fantasma estava assustado, mas Pac-Man não está mais
                ghost.is_frightened = False
                ghost.current_speed = GHOST_SPEED_NORMAL # Retorna à velocidade normal
                print(f"Fantasma {ghost.name} deixou de estar assustado!")
        # Se o fantasma está is_eaten, ele permanece no estado de is_eaten (velocidade normal, cor cinza)
    # 7.3. Desenho na Tela
    screen.fill(BLACK) # Limpa a tela com preto

    # Desenhar o labirinto
    # A lógica de desenho das paredes será mais complexa
    for row_index, row in enumerate(level):
        for col_index, cell in enumerate(row):
            x = col_index * CELL_SIZE
            y = row_index * CELL_SIZE

            # Coordenadas do centro da célula
            center_x = x + CELL_SIZE // 2
            center_y = y + CELL_SIZE // 2

            if cell == 1: # Se a célula é uma parede
                # Desenhar as linhas da parede com base nos vizinhos
                # Cor das paredes
                current_wall_color = BLUE

                # 1. Linhas Horizontais
                # Desenha a linha superior da parede se não houver parede acima
                if row_index > 0 and level[row_index - 1][col_index] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x, y + WALL_LINE_THICKNESS // 2), 
                                     (x + CELL_SIZE, y + WALL_LINE_THICKNESS // 2), 
                                     WALL_LINE_THICKNESS)
                # Desenha a linha inferior da parede se não houver parede abaixo
                if row_index < len(level) - 1 and level[row_index + 1][col_index] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x, y + CELL_SIZE - WALL_LINE_THICKNESS // 2), 
                                     (x + CELL_SIZE, y + CELL_SIZE - WALL_LINE_THICKNESS // 2), 
                                     WALL_LINE_THICKNESS)
                # Desenha linha horizontal se esta parede se conecta a outra parede na mesma linha
                if col_index < len(row) - 1 and level[row_index][col_index + 1] == 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (center_x, center_y), 
                                     (center_x + CELL_SIZE, center_y), 
                                     WALL_LINE_THICKNESS)

                # 2. Linhas Verticais
                # Desenha a linha esquerda da parede se não houver parede à esquerda
                if col_index > 0 and level[row_index][col_index - 1] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x + WALL_LINE_THICKNESS // 2, y), 
                                     (x + WALL_LINE_THICKNESS // 2, y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                # Desenha a linha direita da parede se não houver parede à direita
                if col_index < len(row) - 1 and level[row_index][col_index + 1] != 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (x + CELL_SIZE - WALL_LINE_THICKNESS // 2, y), 
                                     (x + CELL_SIZE - WALL_LINE_THICKNESS // 2, y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                # Desenha linha vertical se esta parede se conecta a outra parede na mesma coluna
                if row_index < len(level) - 1 and level[row_index + 1][col_index] == 1:
                    pygame.draw.line(screen, current_wall_color, 
                                     (center_x, center_y), 
                                     (center_x, center_y + CELL_SIZE), 
                                     WALL_LINE_THICKNESS)
                
                # --- Lógica para Cantos e Junções (Simplificado, pode ser melhorado) ---
                # A lógica acima desenha os segmentos retos. Cantos (onde linhas se encontram
                # em L, T, cruz) precisarão de ajustes ou de algoritmos mais robustos
                # para parecerem perfeitos. Para um visual de néon, muitas vezes você desenha
                # círculos nos cantos ou usa técnicas de arredondamento.

                # Desenha um pequeno círculo no centro de cada parede.
                # Isso pode ajudar a "preencher" a junção das linhas e dar um efeito arredondado.
                pygame.draw.circle(screen, current_wall_color, (center_x, center_y), WALL_LINE_THICKNESS // 2)


            elif cell == 2: # Pílula
                pygame.draw.circle(screen, WHITE, (x + CELL_SIZE // 2, y + CELL_SIZE // 2), CELL_SIZE // 4)
            elif cell == 3: # Pílula de Poder
                pygame.draw.circle(screen, WHITE, (x + CELL_SIZE // 2, y + CELL_SIZE // 2), CELL_SIZE // 2)
            elif cell == 4: # Porta da casa dos fantasmas (pode manter como um retângulo fino)
                pygame.draw.rect(screen, (100, 100, 100), (x, y + CELL_SIZE // 2 - 2, CELL_SIZE, 4))

    # Desenhar o Pac-Man
    pacman.draw(screen)

    # --- Desenhar os Fantasmas ---
    for ghost in ghosts:
        ghost.draw(screen)

    # --- Exibir Pontuação (e futuramente vidas) ---
    if score != last_score_rendered:
        score_text = game_font.render(f"Score: {score}", True, WHITE )  # Renderiza o texto
        last_score_rendered = score

    # Ajuste a posição do texto na tela
    screen.blit(score_text, (5, SCREEN_HEIGHT - 30))  # Exemplo: no canto inferior esquerdo

    # 7.4. Atualizar a Exibição (Mostrar tudo que foi desenhado)
    pygame.display.flip()

    # 7.5. Controle de FPS (Quadros por Segundo)
    clock.tick(60) # Limita o jogo a 60 FPS


Pac-Man Powered Up! Tempo: 19235
Fantasma blinky ficou assustado e inverteu a direção!
Fantasma pinky ficou assustado e inverteu a direção!
Fantasma inky ficou assustado e inverteu a direção!
Fantasma clyde ficou assustado e inverteu a direção!
Pac-Man Power-Up Acabou!
Fantasma blinky deixou de estar assustado!
Fantasma pinky deixou de estar assustado!
Fantasma inky deixou de estar assustado!
Fantasma clyde deixou de estar assustado!


In [45]:
# 8. Finalização do Pygame (Após o Loop Principal)
pygame.quit()